### Improved neural network training code

This code contains the Network class which handles the defining and training of any neural networks we use. In particular this class contains routines such as feed-forward, back-propagation, stochastic gradient descent, network evaluation and more. As mentioned in the script NeuralNetworks.ipynb, the foundations of this neural network code are mostly based on the online tutorial "Neural Networks and Deep Learning" by Michael A. Nielsen, however this is the __improved version__ of this code where I've added a lot more functionality including convolutional layers, pooling layers, batch normalisation, dropout and more. 

To be clear, the parts ot the code which are based on this tutorial were:
* the initialisation of weights and biases for networks;
* the feedforward and backpropagation routines for fully connected layers;
* the organisation of training examples into mini-batches;
* evaluation of the network's performance on test data;
* the update step for the weights and biases;
* implementation of L2 regularisation;
* the code for loading MNIST data;

The improved parts of the code that I wrote from scratch include:
* the initialisation of weights and biases for convolutional layers;
* the feedforward and backpropagation routines for convolutional layers;
* the feedforward and backpropagation for pooling layers;
* the initialisation of parameters 'beta' and 'gamma' for batch normalisation;
* the calculation of the global weighted input means and variances used in batch normalisation;
* the feedforward and backproagation routines for batch normalised layers;
* the update step for batch normalisation parameters beta and gamma;
* implementation of dropout regularisation;
* batch_backprop routine for running backpropagation on an entire mini-batch of training examples simultaneously;
* the initialisation of reinforcement learning hyperparameters;
* use of activation functions ReLU and leaky ReLU; 
* function for visualising convolutional filter weights;
* functions for copying networks and editing network exploration parameter/type;

#### This code is quite long so feel free to skip straight to Results.ipynb.

In [46]:
#Importing standard libraries
import random
import numpy as np
import copy

#Miscellaneous functions mostly used in the Network class.
#Sigmoid activation function.
def sigmoid(z): 
    return 1.0/(1.0+np.exp(-z))

#Derivative of the sigmoid function; the chain rule shows that it can be expressed as follows in terms of itself.
def sigmoid_prime(z):    
    return sigmoid(z)*(1-sigmoid(z))

#Rectified linear unit activation function.
def ReLU(z):     
    return np.maximum(0.0, z)
    
#Derivative of the rectified linear unit function.
def ReLU_prime(z):    
    return 0.5*(np.sign(z)+1)

#Leaky ReLU function; the gradient in the negative region can be chosen but here I always use a gradient of 0.1 to keep things simple.
def leaky_ReLU(z, gradient=1/10):     
    return np.maximum(gradient*z, z)
        
#Derivative of the leaky ReLU function.
def leaky_ReLU_prime(z, gradient=1/10):   
    #If we assume our derivative is of the form a*(sign(z)+b) then a and b can be solved in terms of the assigned gradient as follows:
    b = 2/(1-gradient) - 1
    a = 1/(1+b)
    return a*(np.sign(z)+b)
    
#Function that takes an integer j and returns a 10-entry vector with a 1 in the j^th position and 0 everywhere else.
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [47]:
#Assorted functions made use of in convolutional layers, pooling layers and batch normalisation.

#Here we import some mathematical operations used in the convolutional layers.
#These typically run faster than the equivalent scripts I made for performing these operations so to speed things up I use them wherever I can.
from scipy.signal import convolve2d, correlate2d  #Importing functions to perform efficent 2D convolution and cross-correlation.
from scipy.linalg import block_diag #Importing a function to quickly construct block diagonal matrices.
from skimage.measure import block_reduce #Importing a function to perform efficient max-pooling.

#Function for calculating the expected output shape of the array resulting from the convolution of our 'input layer' with a filter whose dimensions are given.
def convolution_output_shape(input_layer_dimensions, filter_dimensions, padding):
    #Whether or not we use padding will effect the number of outputs of our convolution and thus the number of weights our convolutional layer needs. Note that with padding on we will require more weights.
    if input_layer_dimensions[0] < filter_dimensions[0]: (input_layer_dimensions, filter_dimensions) = (filter_dimensions, input_layer_dimensions)   #If the filter is larger than the layer then the role of the two are swapped in the convolution operation, hence we swap their roles here too.
    if padding == True: output_shape = [input_layer_dimensions[0]+filter_dimensions[0]-1, input_layer_dimensions[1]+filter_dimensions[1]-1] 
    if padding == False: output_shape = [input_layer_dimensions[0]-filter_dimensions[0]+1, input_layer_dimensions[1]-filter_dimensions[1]+1]
    return output_shape

#Function for determining the amount of padding needed in our max-pooling process.
def pooling_padding_needed(input_layer_dimensions, pooling_filter_dimensions, stride):
    padding_dimensions = [0,0]
    for i in [0, 1]:
        padding_dimensions[i] = stride*np.ceil((input_layer_dimensions[i]-pooling_filter_dimensions[i])/stride) - input_layer_dimensions[i] + pooling_filter_dimensions[i]
    return padding_dimensions

#Function for performing max-pooling on a given array.
def max_pooling(input_array, pooling_size, stride, return_indices = False):
    #If we have a stride which doesn't match our pooling size or if we want to keep track of each maximal index during our pooling step we can't simply use the block_reduce function so we need to max pool manually.
    input_array_dimensions = np.shape(input_array)  #Finding the shape of the input array.
    padding_dimensions = pooling_padding_needed(input_array_dimensions, (pooling_size, pooling_size), stride) #Calculating the amount of padding we will need.
    if return_indices == True or stride != pooling_size:
        #Calculating the array size we will need for our pooled output.
        output_shape = max_pooling_output_shape(input_array_dimensions, pooling_size, stride)
        #Creating the padded array and inserting our input_array into the top left corner.
        padded_array = np.zeros((int(input_array_dimensions[0]+padding_dimensions[0]), int(input_array_dimensions[1]+padding_dimensions[1])))
        padded_array[0:input_array_dimensions[0], 0:input_array_dimensions[1]] = input_array
        #Performing the max-pooling process.
        output_array = np.zeros(output_shape)
        indices = []
        for i in range(output_shape[0]):  #Looping over y indices. 
            for j in range(output_shape[1]):   #Looping over x indices. 
                #Defining the current window that our pooling algorithm is going to max-pool over.
                pooling_window = padded_array[i*stride:i*stride+pooling_size, j*stride:j*stride+pooling_size]
                max_index = pooling_window.argmax()
                #Calculating the current row and column of the maximal index in our pooling window.
                row = max_index//pooling_size
                column = max_index%pooling_size
                max_value = pooling_window[row, column]
                output_array[i, j] = max_value
                if max_value == 0: indices.append(None)   #We don't append any values at padded indices (these will always be zero).
                else: indices.append([i*stride+row, j*stride+column]) #Here we save the original indices of the entries that survives max-pooling.
        return output_array, indices
    else:  #In the special case that our stride matches our pooling size we can use the block_reduce function from skimage which typically runs faster.
        output_array = block_reduce(input_array, (pooling_size, pooling_size), np.max)
        return output_array
    
#Equation for calculating the expected output shape of an array with a given input shape after max-pooling using a certain pooling size and stride.
def max_pooling_output_shape(input_array_dimensions, pooling_size, stride):
    padding_dimensions = pooling_padding_needed(input_array_dimensions, (pooling_size, pooling_size), stride)
    output_shape = (int((input_array_dimensions[0]+padding_dimensions[0]-pooling_size+stride)/stride), int((input_array_dimensions[1]+padding_dimensions[1]-pooling_size+stride)/stride))
    return output_shape

#Code for batch normalising a single layer of weighted inputs mb_z from a mini-batch of size m.
def batch_normalise(mb_z, eps, gamma, beta, m): 
    #Reshaping the mini-batch weighted inputs to allow us to easily calculate the mean and variance over the mini-batch.
    mb_z = np.reshape(mb_z, (m, np.size(gamma))).T
    #Calculating the mean and variance of the weighted inputs.
    means = np.mean(mb_z, 1, keepdims=True)
    variances = np.var(mb_z, 1, keepdims=True)
    #Using the mini-batch mean and variance we normalise the weighted inputs.
    normalised_mb_z = (mb_z-means)/np.sqrt(variances+eps)
    normalised_mb_z = np.reshape(normalised_mb_z.T, (np.size(normalised_mb_z), 1))
    #Finally we shift and scale these weighted inputs using our network's learned gamma and beta parameters.
    mb_gamma = np.tile(gamma, (m,1)) 
    mb_beta = np.tile(beta, (m,1))
    final_mb_z = mb_gamma*normalised_mb_z+mb_beta
    return final_mb_z, normalised_mb_z, means, variances  

In [ ]:
#Libraries for importing data sets.
import pickle  
import gzip

#Code for loading the MNIST data into the python runtime. 
def load_data():
    #Opening the file containing the MNIST data.
    f = gzip.open('mnist.pkl.gz', 'rb')
    #Loading the 50,000 training examples and the 10,000 validation and test examples.
    training_data, validation_data, test_data = pickle.load(f, encoding='bytes')
    f.close()
    return (training_data, test_data) #Here I don't use validation data and just return the training and test data.

#Code for organising the training and test data so that its ready for training use.
def load_data_wrapper():
    #Loading in the MNIST training data and test data using our load_data() routine above.
    tr_d, te_d = load_data()
    #Organising the 50,000 training data images from MNIST with their corresponding labels.
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    #For the test data we save the labels in vector form rather than as integers.
    training_results = [vectorized_result(y) for y in tr_d[1]] 
    training_data = list(zip(training_inputs, training_results))
    #Organising the 10,000 test data images from MNIST with their corresponding labels.
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, test_data)

In [ ]:
#Network class: this is the heart of the code as it contains all functions used in defining and training the neural networks.
#This class contains routines such as feed-forward, back-propagation, stochastic gradient descent, network evaluation and more.
#The inputs to this class are the hyperparameters for the neural network which are each explained inside the __init__ routine.
#The `layers' input is simple for fully connected networks; the entries of layers denote the number of nodes you want in each layer.
#For convolutional networks, 'layers' becomes more complicated as more values need to be prescribed so this is best understood by looking at examples in the results.
class Network(object):
    def __init__(self, layers, activation_function = sigmoid, padding = False, dropout_frequency = 0, batch_normalisation = False, data_augmentation = True, game = None, exploration_type = "epsilon-greedy", exploration_parameter = 0):
        
        #Initialising general neural network hyperparameters:
        self.num_layers = len(layers) #Number of layers.
        self.layers = layers #Data used for describing each layer.
        self.padding = padding #Here we set a flag saying whether or not to use zero-padding in any convolutional steps. 
        self.activation_function = activation_function #Activation functions can be sigmoid, ReLU or leaky_ReLU.
        #Assigning the derivatives of the chosen activation functions.
        if activation_function == ReLU: self.activation_function_prime = ReLU_prime
        if activation_function == sigmoid: self.activation_function_prime = sigmoid_prime
        if activation_function == leaky_ReLU: self.activation_function_prime = leaky_ReLU_prime
        self.batch_normalisation = batch_normalisation #Flag saying whether or not to use batch normalisation.
        self.dropout_frequency = dropout_frequency #The rate at which nodes are dropped in all layers (except the input layer).
        
        #Initialising reinforcement learning exclusive hyperparameters:
        self.game = game #The game to be trained on can be "connect 4" or "noughts and crosses".
        self.exploration_type = exploration_type #Exploration types can be "epsilon-greedy", "softmax" or "adaptive-greedy".
        self.exploration_parameter = exploration_parameter #Exploration parameter; for "epsilon-greedy this represents epsilon, otherwise this represents the exploration temperature T.
        self.data_augmentation = data_augmentation #Whether or not data augmentation is used to generate new experiences by flipping/rotating boards.
        #Creating an array for storing a network's experiences in reinforcement learning.
        self.experiences = []
        
        #Initialising storage for our weights and biases.
        self.weights = [None for layer in layers[:-1]]  
        self.biases = [None for layer in layers[1:]]
        
        #Batch normalisation initialisations.
        if batch_normalisation == True:
            #Initialising storage for our gamma and beta variables.
            self.gammas = [None for layer in layers[:-2]] 
            self.betas = [None for layer in layers[:-2]] 
            self.epsilon = 1e-4 #Value of epsilon added to the variance in the division part of the batch normalisation step (not to be confused with epsilon in epsilon-greedy exploration which is saved under `exploration_parameter').
            #Setting default values of the global mean and the inverse global variance used for feedforward in evaluation and gameplay (these are updated during learning).
            self.global_mean = 0 
            self.inv_sqrt_global_variance = 1 
            
        #Here we don't allow our final layer to be convolutional as it needs to have a node for every choice. Typically this isn't a strict requirement for a network but I'm enforcing it here to make things slightly simpler
        if type(layers[-1]) == list:
            print("Final layer must not be a convolutional layer.")
            return
        
        #Looping over each layer in our network to initialise weights and biases in each layer.
        for i in range(self.num_layers-1):
            
            #Initialising parameters for convolutional layers.
            #We can describe a convolutional layer in our network by entering a 6 component list entry in place of the usual 'layer_size' entry in our layers array.
            #The 6 components are: [input_layer_rows, input_layer_columns, num_filters, filter_size (filters are assumed to be square), pooling_size, pooling_stride]
            #Hence we need to describe the shape that our input activations should be rearranged into using input_layer_rows and input_layer_columns.
            if type(layers[i]) == list:   
                #Here we make sure that the correct data is given to describe our convolutional layer.
                if len(layers[i]) < 4:   
                    print("Convolutional layer is missing required arguments: input_layer_rows, input_layer_columns, num_filters, filter_size. Optional arguments also include pooling_size, pooling_stride (with default values 1).")
                    return
                if len(layers[i]) > 6:
                    print("Too many input arguments for convolutional layer. Allowed input arguments are: input_layer_rows, input_layer_columns, num_filters, filter_size, pooling_size, pooling_stride.")
                    return
                #If we don't specify a pooling size or a pooling stride then we set these to have value 1 by default.
                while len(layers[i]) < 6:  
                    layers[i].append(1)
                    
                #Unpacking our convolutional layer information.
                [input_layer_rows, input_layer_columns, num_filters, filter_size, pooling_size, pooling_stride] = layers[i]  
                input_layer_size = input_layer_rows*input_layer_columns
                #In the convolutional layer we need a bias for every node in the layer and we also need filter_size weights per filter as well as a next layer bias for every filter.
                #Our weights are randomly initialised using a normal distribution with mean 0 and s.d. given by 1/sqrt(no. of input weights to node) for each node.
                #This means that our weighted input 'z' into the node is much more likely to be a small number (around -1 to 1 usually) which avoids large activation values and reduces node saturation.
                self.biases[i] = np.array(np.random.randn(num_filters, 1),dtype=np.float32) #Here np.random.randn(x, 1) makes a x-by-1 array of random numbers.
                self.weights[i] = np.array(np.random.randn(num_filters, filter_size*filter_size)/np.sqrt(input_layer_size),dtype=np.float32)  #Here we normalise by our filter size in each filter.
                #We need to ensure that the output of our convolutional layer fits the input of the next layer. Note that whether or not we use padding will effect the number of outputs of our convolutional layer.
                layer_dimensions, filter_dimensions = [input_layer_rows, input_layer_columns], [filter_size, filter_size]
                #Determining the size we will need for the layer following our convolutional layer. 
                conv_layer_output_size = convolution_output_shape(layer_dimensions, filter_dimensions, padding)
                if pooling_size > 1:  #If we are adding pooling after our convolution then the size of the output will also depend on the pooling size and stride.
                    for j in [0, 1]:
                        conv_layer_output_size[j] = 1 + np.ceil((conv_layer_output_size[j]-pooling_size)/pooling_stride)
                conv_layer_output_size = int(num_filters*conv_layer_output_size[0]*conv_layer_output_size[1])
                #Checking to see whether the current layer is compatible with the output of the previous convolutional layer.
                if type(layers[i+1]) == list: next_layer_input_size = layers[i+1][0]*layers[i+1][1]
                else: next_layer_input_size = layers[i+1]
                #If the output of a convolutional layer doesn't match the input of the next layer then we print out an error message and return.
                if conv_layer_output_size != next_layer_input_size:
                    print("A layer after a convolutional layer doesn't match the size of the expected convolutional layer output: ",next_layer_input_size,"!=",conv_layer_output_size)
                    return
            
            #Initialising parameters for fully connected layers.
            else:
                if type(layers[i+1]) == list: 
                    next_layer_size = layers[i+1][0]*layers[i+1][1]  #Determining the size needed for inputs to our layer if it's a convolutional layer.                    
                else: next_layer_size = layers[i+1]  #Otherwise we have a fully connected layer so its size is simply prescribed by the value of layers[i+1].
                self.biases[i] = np.array(np.random.randn(next_layer_size, 1),dtype=np.float32) #Note that we don't define biases for fully connected first layer nodes.
                self.weights[i] = np.array(np.random.randn(next_layer_size, layers[i])/np.sqrt(layers[i]),dtype=np.float32) 
                #Initialising batch_normalisation parameters.
                if batch_normalisation == True and i != self.num_layers-2: 
                    self.gammas[i] = np.array(np.random.randn(layers[i], 1),dtype=np.float32) #Randomly initialising gamma values for all layers but our last two layers.
                    self.betas[i] = np.array(np.random.randn(layers[i], 1),dtype=np.float32) #Randomly initialising gamma values for all layers but our last two layers.

    #Code for feeding forward an array of input activations 'a' through our network to produce output activations.
    def feedforward(self, a):
        #Calculations for batch normalisation of our input activations (if batch_normalisation is enabled).
        if self.batch_normalisation == True:
            gamma = self.gammas[0]
            beta = self.betas[0]
            if self.global_mean is 0: #If we are feeding forward without having done any training we don't have any global means/variances, hence a is already normalised.
                normalised_a = a 
            else: #Otherwise we normalise our input activations using the saved global mean and global variance from all examples.
                normalised_a = (a-self.global_mean[0])*self.inv_sqrt_global_variance[0]
            #Scaling and shifting the normalised activations using the learned parameters gamma and beta.
            a = gamma*normalised_a+beta  
            
        #Here we loop over each layer in our network.
        layer = 0 #Current layer counter.
        for b, w in zip(self.biases, self.weights):   #The arrays of both the bias vectors and weight matrices have length num_layers-1 since there are no first layer biases and no final layer weights.
           
            #Convolutional layer calculations.
            if type(self.layers[layer]) == list:   #If the current layer is described by a list then it is a convolutional layer so we perform a convolution to get the activation.
                [input_layer_rows, input_layer_columns, num_filters, filter_size, pooling_size, pooling_stride] = self.layers[layer]  #Unpacking our convolutional layer information.
                 #We convolve (or technically cross-correlate) each of our filters with 2D array of activations fed into our convolutional layer by our previous layer to get a weighted input for each filter.
                reshaped_a = np.reshape(a, (input_layer_rows, input_layer_columns))
                a = np.array((),dtype=np.float32)
                #Calculating convolutions using each different filter.
                for j in range(num_filters):  #The weights for each filter are stored in separate vectors contained in the entry of 'weights' corresponding to the convolutional layer.
                    convolution_filter = np.reshape(w[j], (filter_size, filter_size))
                    #If we have padding enabled then we obtain an outputs that include the cases where our filter isn't entirely overlapping with our 2D array of nodes, i.e. when it's hanging off the edge.
                    if self.padding == True: z = correlate2d(reshaped_a, convolution_filter, 'full') + b[j]  
                    #Otherwise we only obtain outputs from calculations made when our filter exactly fits onto our 2D array of nodes.
                    if self.padding == False: z = correlate2d(reshaped_a, convolution_filter, 'valid') + b[j] 
                     #If a pooling filter size is prescribed we then apply max-pooling to the output of the convolution of our filter with our input activations.
                    if pooling_size > 1: z = max_pooling(z, pooling_size, pooling_stride)
                    #Finally we apply our activation function to the pooled output and append this to our array of activations. 
                    a = np.append(a, self.activation_function(z))   
                a = a.flatten()[:, np.newaxis]
            
            #Fully connected layer calculations.
            else:
                #Batch normalisation step: here we need to normalise our weighted inputs for each layer and scale them using gamma and beta. 
                #During the testing phase we normalise using the mean and standard deviation from the whole training set (rather than from a mini-batch).
                if self.batch_normalisation == True and layer < self.num_layers-3:
                    z = w@a
                    gamma = self.gammas[layer+1]
                    beta = self.betas[layer+1]
                    if self.global_mean is 0: normalised_z = z #Accounting for when we are feeding forward without having done any training (so we don't have any global means/variances).
                    else: normalised_z = (z-self.global_mean[layer+1])*self.inv_sqrt_global_variance[layer+1]
                    z = gamma*normalised_z+beta 
                else: #Fully connected layer calculations.
                    z = w@a+b
                #If we are training our network to play games then we always use a linear final layer as estimating action-values is a regression problem.
                if layer == self.num_layers-2 and self.game != None: a = z
                #Otherwise we apply our activation function to each weighted input to introduce non-linearity into our network.
                else: a = self.activation_function(z)                            
            layer = layer + 1  #Incrementing our layer counter.
        return a

    
    #Function for training a neural network using stochastic gradient descent for a given number of epochs.
    #Here eta is the learning rate and lmbda is the hyperparameter for L2 regularisation.
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data = None, lmbda = 0):
        
        #Checking that we have an appropriate mini-batch size if batch normalisation is being used.
        if self.batch_normalisation == True and mini_batch_size == 1:
            print("In order for batch normalisation to be used, mini_batch_size must be greater than 1.")
            return
        #Recording the size of the test and training data and performing an initial evaluation on the untrained network.
        if test_data: 
            n_test = len(test_data)
            correct = self.evaluate(test_data)
            accuracies = [100*correct/n_test]
        n = len(training_data) 
        
        #Looping over each epoch.
        for j in range(epochs):
            #Creating the mini-batches.
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            global_mean, global_variance = 0, 0
            for mini_batch in mini_batches:
                #Performing a single gradient descent step for each mini-batch.
                mb_means, mb_variances = self.mini_batch_gradient_descent(mini_batch, eta, lmbda, n) #If we aren't using batch normalisation then means and variances will return as None.
                if self.batch_normalisation == True:
                    global_mean = np.add(global_mean, mb_means)
                    global_variance = np.add(global_variance, mb_variances)
            #If we are using batch normalisation then we need to calculate the mean and variance of our entire training set for use in the testing phase later.
            #We calculate the population mean/variance using averages from all of the mini-batch means/averages calculated during training.
            if self.batch_normalisation == True:
                inv_len_mini_batches = 1/len(mini_batches)
                self.global_mean = global_mean*inv_len_mini_batches
                global_variance = mini_batch_size/(mini_batch_size-1)*global_variance*inv_len_mini_batches
                if np.size(global_variance) > 1: self.inv_sqrt_global_variance = [1/np.sqrt(gv+self.epsilon) for gv in global_variance]
            #If test data is provided then the network will evaluate its performance against this data after every epoch.
            if test_data:
                correct = self.evaluate(test_data)
                accuracies.append(100*correct/n_test)
        if test_data: return accuracies
    
    #This code tweaks the network's learnable parameters using gradient descent over a single mini-batch.
    def mini_batch_gradient_descent(self, mini_batch, eta, lmbda, n):
        #Initialising storage for the gradients of our cost function C wrt the network's weights and biases.
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        #Defining a coefficient used in our weight update step
        w_coeff = 1-eta*(lmbda/n)
        inv_len_mini_batch = 1/len(mini_batch)
        #All our gradient terms below (e.g. grad_w, grad_b) use the below coefficient in our update step.
        grad_coeff = eta*inv_len_mini_batch 
        convolutional = False
        for layer in self.layers:         #Determining whether our network has any convolutional layers.
            if type(layer) == list: convolutional = True
           
        #Calculations without batch normalisation.
        if self.batch_normalisation == False: 
            #We only use batch backpropagation if our mini-batch size is greater than 1 and if our network has no convolutional layers.
            if convolutional == False and len(mini_batch) > 1: grad_b, grad_w = self.batch_backprop(mini_batch) 
            else: 
                for x, y in mini_batch:
                    delta_grad_b, delta_grad_w = self.backprop(x, y)
                    grad_b = [gb+dgb for gb, dgb in zip(grad_b, delta_grad_b)]
                    grad_w = [gw+dgw for gw, dgw in zip(grad_w, delta_grad_w)]
            #Update step for our learnable parameters.
            self.weights = [w_coeff*w-grad_coeff*gw for w, gw in zip(self.weights, grad_w)]
            self.biases = [b-grad_coeff*gb for b, gb in zip(self.biases, grad_b)]  
            return None, None
        
        #Batch normalisation calculations.
        if self.batch_normalisation == True: 
            grad_gamma, grad_beta, grad_b, grad_w, mb_means, mb_variances = self.batch_backprop(mini_batch)
            #Update step for our learnable parameters.
            self.biases = [b-grad_coeff*gb for b, gb in zip(self.biases, grad_b)]  
            self.weights = [w_coeff*w-grad_coeff*gw for w, gw in zip(self.weights, grad_w)]
            self.gammas = [gamma-grad_coeff*ggamma for gamma, ggamma in zip(self.gammas, grad_gamma)]
            self.betas = [beta-grad_coeff*gbeta for beta, gbeta in zip(self.betas, grad_beta)]
            return mb_means, mb_variances

    #Function for back-propagation through our network taking in one mini batch tuple input (x,y) at a time to calculate the gradient
    #vectors grad_b and grad_w of our cost function C wrt our weights and biases.
    def backprop(self, x, y): 
        #Initialising arrays to store our partial derivatives of C wrt weights and biases.
        grad_b = [np.array(np.zeros(np.shape(b)),dtype=np.float32) for b in self.biases] 
        grad_w = [np.array(np.zeros(np.shape(w)),dtype=np.float32) for w in self.weights] 
        #Here we feedforward to calculate our weighted inputs z^l and our activations a^l := activation_function(z) for each layer l.
        #Using these we can then calculate the final layer error vector delta^L and the other layer error vectors delta^l for all l.
        activation = x #This sets our mini batch inputs as our first layer activation vector.
        activations = [activation] #This creates a matrix to store each of our layer activations.
        zs = [] #This creates an array to store all of our weighted input vectors z (or z^l).
        max_pooling_indices = [] #This creates an array to keep track of any maximal indices used in any max pooling steps we perform (we need these later for backpropagtion).
        
        #Dropout calculations
        if self.dropout_frequency > 0: 
            #If we are using dropout then we create a 'mask' for each hidden layer which tell us which neurons to drop in each layer (this will vary per training example).
            masks = [] #Creating an array to hold the mask for each layer.
            for l in range(self.num_layers-3): #In this case we don't perform dropout on the first (input) layer, final (output) layer or the penultimate layer so we only create (up to) num_layers-3 masks.
                if type(self.layers[l+1]) == list: masks.append(None) #We won't perform dropout on any convolutional layers. 
                else: masks.append(np.random.binomial(1, 1-self.dropout_frequency, (self.layers[l+1], 1))/(1-self.dropout_frequency)) #We divide by the dropout frequency since we are using 'inverted dropout'.       
        
        #Feed-forward part of our backpropagation routine.
        layer = 0
        for b, w in zip(self.biases, self.weights):   #The arrays of both the bias vectors and weight matrices have length num_layers-1 since there are no first layer biases and no final layer weights.
            max_z_indices = []
            
            #Convolutional layer calculations.
            if type(self.layers[layer]) == list:   #If the current layer is described by a list then it is a convolutional layer so we perform a convolution to get the activation.
                [input_layer_rows, input_layer_columns, num_filters, filter_size, pooling_size, pooling_stride] = self.layers[layer]  #Unpacking our convolutional layer information.
                 #We convolve (or technically cross-correlate) each of our filters with 2D array of activations fed into our convolutional layer by our previous layer to get a weighted input for each filter.
                reshaped_activation = np.reshape(activation, (input_layer_rows, input_layer_columns))
                z = np.array((),dtype=np.float32)
                activation = np.array((),dtype=np.float32)
                #Calculating the convolution for each filter.
                for f in range(num_filters):  #The weights for each filter are stored in separate vectors contained in the entry of 'weights' corresponding to the convolutional layer.
                    convolution_filter = np.reshape(w[f], (filter_size, filter_size))  
                    if self.padding == True: z_f = correlate2d(reshaped_activation, convolution_filter, 'full') + b[f]  #If we have padding enabled then we obtain an outputs that include the cases where our filter isn't entirely overlapping with our 2D array of nodes, i.e. when it's hanging off the edge.
                    if self.padding == False: z_f = correlate2d(reshaped_activation, convolution_filter, 'valid') + b[f] #Otherwise we only obtain outputs from calculations made when our filter exactly fits onto our 2D array of nodes.
                    #Max-pooling calculations.
                    if pooling_size > 1: 
                        z_f, max_z_f_indices = max_pooling(z_f, pooling_size, pooling_stride, True) #If a pooling filter size is prescribed we then apply max-pooling to the output of the convolution of our filter with our input activations.
                        max_z_indices.append(max_z_f_indices)
                    z = np.append(z, z_f)  #Appending the weighted input from this filter to our overall weighted input.
                    activation = np.append(activation, self.activation_function(z_f))   #We then apply our activation function to the output obtained by convolving our filter with our input activations and then append this to the array of activations.
                z = z.flatten()[:, np.newaxis]  #Reshaping our overall z and activation into the format that works with our backpropagation algorithm.
                activation = activation.flatten()[:, np.newaxis]
            
            #Fully connected layer calculations.
            else:
                z = w@activation+b   #Calculating our weighted input (for a non-convolutional layer).
                #If we are training our network to play games then we always use a linear final layer as estimating action-values is a regression problem.
                if layer == self.num_layers-2 and self.game != None: activation = z   
                #Here we apply our activation function which introduces non-linearity into our network.
                else: activation = self.activation_function(z)  
            activations.append(activation)
            
            #We never apply dropout to our output layer or the penultimate layer. Also the saved activations for backpropagation aren't the dropout activations since the backpropagation dropout is achieved by applying our mask to the delta passed back to the dropout layer.
            if self.dropout_frequency > 0 and layer < self.num_layers-3: 
                if masks[layer] is not None: 
                    activation = activation * masks[layer] #Applying dropout to hidden layers on the forward pass.
            zs.append(z)                    
            max_pooling_indices.append(max_z_indices)
            layer = layer + 1
            
        #Backpropagation part of our backprop routine.
        #First we calculate the error in the final layer using our output activations and the derivative of our cost function.
        if self.game != None: delta = self.cost_derivative(activations[-1], y) #If we are training for reinforcement learning then this 
        else: delta = self.cost_derivative(activations[-1], y)*self.activation_function_prime(zs[-1]) #Calculates our final layer error vector.

        #Next we backpropagate one layer at a time to calculate the error delta for each layer and then we calculate our cost gradient vectors grad_b and grad_w for each layer.
        #First we use our last layer delta to calculate the next delta and pass it on: since the last layer is forced to not be convolutional this is just our standard delta calculation.
        for l in range(1, self.num_layers): 
            if l != self.num_layers-1: z = zs[-l-1] #Unless we are on the first layer we will need our previous layer weighted input at each step.
            
            #Convolutional layer calculations.
            if type(self.layers[-l-1]) == list:  
                #Using the current layer delta to calculate grad_b and grad_w for a convolutional layer.
                [input_layer_rows, input_layer_columns, num_filters, filter_size, pooling_size, pooling_stride] = self.layers[-l-1]  #Unpacking the data for our convolutional layer.
                #Calculating the delta to be passed down to the next layer; note that as our current layer is a convolutional layer the calculations are more complicated than before.
                layer_dimensions, filter_dimensions = [input_layer_rows, input_layer_columns], [filter_size, filter_size]
                delta_f_sizes = convolution_output_shape(layer_dimensions, filter_dimensions, self.padding)  #Calculating the dimension that each delta_f array should have when reshaped.
                unpooled_delta_f_sizes = delta_f_sizes
                #Pooling calculations.
                if pooling_size > 1:   #If pooling was used then the dimensions in delta_f_sizes will be smaller than as calculated above.
                    padding_dimensions = pooling_padding_needed(delta_f_sizes, (pooling_size, pooling_size), pooling_stride)
                    delta_f_sizes = (int((delta_f_sizes[0]+padding_dimensions[0]-pooling_size+pooling_stride)/pooling_stride), int((delta_f_sizes[1]+padding_dimensions[1]-pooling_size+pooling_stride)/pooling_stride))
                delta_f_size = delta_f_sizes[0]*delta_f_sizes[1]
                delta_fs = []  #Creating an array to store the delta arrays for each filter.
                reshaped_activation = np.reshape(activations[-l-1], (input_layer_rows, input_layer_columns))
                for f in range(num_filters):  #For convolutional layers we build up our gradients using the deltas for each filter (i.e. the sections of the overall delta).
                    convolution_filter = np.reshape(self.weights[-l][f], (filter_size, filter_size))  
                    delta_f = np.reshape(delta[f*delta_f_size:(f+1)*delta_f_size], (delta_f_sizes[0], delta_f_sizes[1]))
                    #If pooling has been used then we essentially need to undo our pooling step by increasing the size of delta_f and inserting zeros at the entries of the non-maximal elements.
                    if pooling_size > 1:   #Pooling layer delta_f calculations; here we only have non-zero entries of delta_f corresponding to the activations which were maximal in the max pooling step.
                        z_f_shape = convolution_output_shape((input_layer_rows, input_layer_columns), (filter_size, filter_size), self.padding)
                        unpooled_delta_f = np.zeros(z_f_shape)   #Initialising an array the size of our convolutional layer's convolution output.
                        max_indices = max_pooling_indices[-l][f]
                        k = 0  #Counter to move through the max_indices array.
                        for i in range(delta_f_sizes[0]):    #Looping over rows.
                            for j in range(delta_f_sizes[1]):   #Looping over columns.
                                if max_indices[k] is not None:
                                    unpooled_delta_f[max_indices[k][0],max_indices[k][1]] = delta_f[i,j]
                                k = k + 1
                        delta_f = unpooled_delta_f
                    grad_b[-l][f] = sum(delta_f.flatten())   #Calculating grad_b for the bias associated with filter f.
                    grad_w[-l][f] = correlate2d(reshaped_activation, delta_f, 'valid').flatten()   #Calculating grad_w for the weights associated with filter f.
                    #We obtain a contribution to the error to be passed to our next layer by convolving the individual errors for each filter (i.e. each delta_f) with that filter.
                    if l != self.num_layers-1:
                        next_delta_f_sizes = convolution_output_shape(np.shape(convolution_filter), unpooled_delta_f_sizes, True)
                        #We use convolution to calculate the next delta to be passed down for each filter.
                        delta_f = convolve2d(delta_f, convolution_filter, 'full')*np.reshape(self.activation_function_prime(z), next_delta_f_sizes)
                        delta_fs.append(delta_f)
                #The overall error to be passed down to the next layer is then the elementwise sum of all of the delta_fs. Note that for our last layer we can skip this step as there is no further layer to pass on a delta to.
                if l != self.num_layers-1:   
                    delta = np.zeros(np.shape(delta_fs[0]))
                    for df in delta_fs:
                        delta = delta + df
                    delta = delta.flatten()[:, np.newaxis] 
                    
            #Fully connected layer calculations.
            else:   
                #Using the current layer delta to calculate grad_b and grad_w for a fully connected layer.
                grad_b[-l] = delta
                grad_w[-l] = delta@activations[-l-1].T   #Again this creates a matrix since we are multiplying our vectors lengthways.
                #We then calculate the value of delta to be passed down from this layer to the next. Note that for our last layer we can skip this step as there is no further layer to pass on a delta to.
                if l != self.num_layers-1: delta = (self.weights[-l].T@delta)*self.activation_function_prime(z)  
            #If dropout was used on a layer in our forward pass then we apply dropout to that layer's delta which makes the entries of delta corresponding to any dropped nodes equal to zero.
            if self.dropout_frequency > 0:
                if l != self.num_layers-1 and l != 1: 
                    if masks[-l+1] is not None:
                        delta = delta * masks[-l+1] #Applying dropout to hidden layer deltas on the backward pass.
        return (grad_b, grad_w) #Finally we return a tuple of our cost gradient vectors wrt our biases and matrices wrt our weights for our whole network.
    
    
    #This function performs the feed-forward and back-propagation algorithm on an entire mini-batch at once, working through one layer at a time. This tends to run faster than the regular backprop routine.
    #Note that this is required for batch normalisation (although I didn't make this compatible with convolutional layers because that would have been too much of a headache).
    def batch_backprop(self, mini_batch):        
        #Initialising arrays to store our partial derivatives of C wrt weights and biases.
        if self.batch_normalisation == False:
            grad_b = [np.array(np.zeros(np.shape(b)),dtype=np.float32) for b in self.biases] 
            grad_w = [np.array(np.zeros(np.shape(w)),dtype=np.float32) for w in self.weights]
        mb_activation, mb_y = [], [] #Creating array to hold the activations 
        
        #Concatenating the input activations from our mini-batch into one large activation.
        m = len(mini_batch)
        for x, y in mini_batch:
            if len(mb_activation) == 0: mb_activation = x
            else: mb_activation = np.concatenate((mb_activation, x))
            if len(mb_y) == 0: mb_y = y
            else: mb_y = np.concatenate((mb_y, y))
        final_mb_zs = []
        
        #Batch normalisation calculations.
        if self.batch_normalisation == True:
            #Initialising arrays to store our partial derivatives of C wrt weights and biases.
            grad_w = [np.array(np.zeros(np.shape(w)),dtype=np.float32) for w in self.weights] #Initialises the gradient vector for our weights.
            grad_b = [np.array(np.zeros(np.shape(b)),dtype=np.float32) for b in self.biases] #Initialises the gradient vector for our biases.
            grad_gamma = [] 
            grad_beta = []
            layer_means = []
            layer_variances = []
            normalised_mb_zs = [] #Array to store the normalised and fully batch normalised mini batch weighted inputs after performing batch normalisation.
            #Batch normalising the input activations and saving the means and variances of these activations.
            mb_activation, normalised_mb_z, means, variances = batch_normalise(mb_activation, self.epsilon, self.gammas[0], self.betas[0], m)
            normalised_mb_zs.append(normalised_mb_z)
            layer_means.append(means)  #We save the mean and variance from this layer's batch normalisation for use in backpropagation later.
            layer_variances.append(variances)
        final_mb_zs.append(mb_activation)
        mb_activations = [mb_activation] #This creates a matrix to store each of our layer activations.
        mb_zs = [] #Array to store all of our weighted input vectors z (or z^l).
        max_pooling_indices = [] #This creates an array to keep track of any maximal indices used in any max pooling steps we perform (we need these later for backpropagtion).
        
        #Feedforward part of our batch_backprop routine.
        layer = 0 #Current layer counter.
        for b, w in zip(self.biases, self.weights):   #The arrays of both the bias vectors and weight matrices have length num_layers-1 since there are no first layer biases and no final layer weights.
            max_z_indices = []
            layer_size = np.shape(w)[1]
            mb_z = []
            
            #Batch normalisation calculations (we don't batch normalise the final layer).
            if self.batch_normalisation == True and layer < self.num_layers-3:
                for i in range(m):
                    z = w@mb_activation[i*layer_size:(i+1)*layer_size]   #If we are using batch normalisation then we don't need to use biases (they cancel out in the batch normalisation step).
                    if len(mb_z) == 0: mb_z = z
                    else: mb_z = np.concatenate((mb_z, z))
                mb_zs.append(mb_z)
                gamma = self.gammas[layer+1]
                beta = self.betas[layer+1]
                #Batch normalising this layer of weighted inputs over our mini-batch.
                mb_z, normalised_mb_z, means, variances = batch_normalise(mb_z, self.epsilon, gamma, beta, m)
                #We save the mean, variance and normalised mini-batch weighted inputs from this layer's batch normalisation for use in backpropagation later.
                layer_means.append(means) 
                layer_variances.append(variances)
                normalised_mb_zs.append(normalised_mb_z)
                final_mb_zs.append(mb_z)
                
            #Calculations without batch normalisation.
            else: 
                for i in range(m):
                    #Calculating our weighted input for each example in our mini-batch
                    z = w@mb_activation[i*layer_size:(i+1)*layer_size]+b 
                    if len(mb_z) == 0: mb_z = z
                    else: mb_z = np.concatenate((mb_z, z))
                mb_zs.append(mb_z) 
                final_mb_zs.append(mb_z)
            #If we are training a network to play a game then we don't apply our activation function to our final layer weighted inputs so that our outputs take the form of values.
            if layer == self.num_layers-2 and self.game != None:  mb_activation = mb_z   
            #Here we apply our activation function which introduces non-linearity into our network.
            else: mb_activation = self.activation_function(mb_z)  
            mb_activations.append(mb_activation)
            max_pooling_indices.append(max_z_indices)
            layer = layer + 1
            
        #Backpropagation part of our batch_backprop routine.
        #First we calculate the error in the final layer using our mini-batch output activations and the derivative of our cost function.
        if self.game != None: mb_delta = self.cost_derivative(mb_activations[-1], mb_y) #If we are using a network to play games then we want to use a linear final layer since predicting the state values is a regression problem.
        else: mb_delta = self.cost_derivative(mb_activations[-1], mb_y)*self.activation_function_prime(mb_zs[-1]) #Calculates our final layer error vector over our whole mini-batch.
        #Here we backpropagate one layer at a time to calculate the error delta for each layer and then we calculate our cost gradient vectors grad_b and grad_w for each layer.
        #First we use our last layer delta to calculate the next delta and pass it on: since the last layer is forced to not be convolutional this is just our standard delta calculation.
        for l in range(1, self.num_layers): 
            if l != self.num_layers-1: mb_z = mb_zs[-l-1] #Unless we are on the first layer we will need our previous layer weighted input at each step.
            #Using the current layer delta to calculate grad_b and grad_w for a fully connected layer.
            w_shape = np.shape(self.weights[-l])
            layer_size = w_shape[1]
            next_layer_size = w_shape[0]
            #Calculating the small changes required for this layer's weights (these are needed regardless of whether we use batch normalisation or not).
            mb_grad_w = np.zeros((m, w_shape[0], w_shape[1]))
            mb_grad_y = mb_delta
            
            #Batch normalisation calculations.
            if self.batch_normalisation == True:
                #If we are using batch normalisation then there are many intermediate calculations we need to do before calculating the previous layer's value of mb_delta.    
                if l > 2: #For all layers but the final two we recieve an mb_delta that represents dC/dy, hence to recover our actual mb_delta=dC/dx we need to use the batch normalisation backprop equations.
                    #First we define some quantities ahead of time that we will be repeatedly using to save time.
                    inv_m = 1/m
                    mb_inv_sqrt_variances = np.tile((layer_variances[-l+2] + self.epsilon)**-0.5, (m,1))
                    #We then calculate mb_delta=dC/dx to be passed down to the next layer.
                    #This involves the calculation of many intermediate quantities according to the chain rule.
                    mb_z = mb_zs[-l]
                    final_mb_z = final_mb_zs[-l+2]
                    mb_means = np.tile(layer_means[-l+2], (m,1))
                    mb_gammas = np.tile(self.gammas[-l+2], (m,1))
                    grad_normalised_mb_z = mb_gammas*mb_grad_y
                    grad_variances = -0.5*np.reshape(np.sum(np.reshape(grad_normalised_mb_z*(mb_z-mb_means)*mb_inv_sqrt_variances**(3), (m, next_layer_size)), 0), (next_layer_size, 1))
                    grad_means = np.sum(np.reshape(-grad_normalised_mb_z*mb_inv_sqrt_variances, (m, next_layer_size)), 0, keepdims=True).T + grad_variances*np.sum(-2*(mb_z-mb_means), 0, keepdims=True)*inv_m
                    mb_grad_means = np.tile(grad_means, (m,1))
                    mb_grad_variances = np.tile(grad_variances, (m,1))
                    grad_mb_z = grad_normalised_mb_z*mb_inv_sqrt_variances + 2*mb_grad_variances*(mb_z-mb_means)*inv_m + mb_grad_means*inv_m
                    mb_delta = grad_mb_z
            
            #We can then calculate our partial derivatives of C wrt our weights and biases for each example in the mini-batch.
            for i in range(m):
                mb_grad_w[i] = mb_delta[i*next_layer_size:(i+1)*next_layer_size]@np.reshape(mb_activations[-l-1].T[0][i*layer_size:(i+1)*layer_size], (1, layer_size))
            reshaped_mb_grad_w = np.reshape(mb_grad_w, (m, next_layer_size, layer_size))
            grad_w[-l] = np.sum(reshaped_mb_grad_w, 0)
            reshaped_mb_delta = np.reshape(mb_delta, (m, next_layer_size))
            grad_b[-l] = np.sum(reshaped_mb_delta, 0)
            grad_b[-l] = grad_b[-l][:, np.newaxis]
            mb_z = final_mb_zs[-l-1]
            new_mb_delta = []
            
            #Calculating the mini-batch error vector mb_delta to be passed down to the next layer.
            for i in range(m): 
                if l == self.num_layers-1:
                    delta = self.weights[-l].T@mb_delta[i*next_layer_size:(i+1)*next_layer_size]    #Calculating our weighted input.
                else:
                    delta = (self.weights[-l].T@mb_delta[i*next_layer_size:(i+1)*next_layer_size])*self.activation_function_prime(mb_z[i*layer_size:(i+1)*layer_size])    #Calculating our weighted input.
                if len(new_mb_delta) == 0: new_mb_delta = delta
                else: new_mb_delta = np.concatenate((new_mb_delta, delta))
            mb_delta = new_mb_delta
            
            #Calculating the derivatives of C wrt our batch normalisation parameters beta and gamma.
            if self.batch_normalisation == True and l > 2:
                reshaped_mb_grad_y = np.reshape(mb_grad_y, (m, next_layer_size))
                grad_beta.insert(0, np.sum(reshaped_mb_grad_y.T, 1, keepdims=True))
                reshaped_normalised_mb_z = np.reshape(normalised_mb_zs[-l+2], (m, next_layer_size))
                grad_gamma.insert(0, np.sum((reshaped_mb_grad_y*reshaped_normalised_mb_z).T, 1, keepdims=True))
        mb_grad_y = mb_delta
        
        #Calculating the derivatives of C wrt our batch normalisation parameters beta and gamma for our input layer. 
        if self.batch_normalisation == True:
            reshaped_mb_grad_y = np.reshape(mb_grad_y, (m, layer_size))
            grad_beta.insert(0, np.sum(reshaped_mb_grad_y.T, 1, keepdims=True))
            reshaped_normalised_mb_z = np.reshape(normalised_mb_zs[0], (m, layer_size))
            grad_gamma.insert(0, np.sum((reshaped_mb_grad_y*reshaped_normalised_mb_z).T, 1, keepdims=True))
            
        #Finally we return a tuple of our cost gradient vectors wrt our biases and matrices wrt our weights for our whole network.
        if self.batch_normalisation == False: return (grad_b, grad_w)
        #If we are using batch normalisation then we instead return the cost gradient vectors wrt our weights, biases, gammas and betas (although the biases go unused).
        if self.batch_normalisation == True: return (grad_gamma, grad_beta, grad_b, grad_w, layer_means, layer_variances) 

    #Function for evaluating a network's performance on unseen test data.
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    #Derivative of the L^2 cost.
    def cost_derivative(self, output_activations, y):
        return (output_activations-y)
    
    #Code for displaying the learned weights of each filter in a network's convolutional layers.
    def display_weights(self, plot = True):
        for l in range(self.num_layers):
            layer = self.layers[l]
            if type(layer) == list:
                filter_size = layer[2]
                i = 1
                for w in self.weights[l]:
                    if plot == True:
                        filter_plot(w, i)
                    else:
                        w = np.reshape(w, (filter_size, filter_size))
                        print("Filter ",i," weights:")
                        print(w)
                        print("")
                    i = i + 1
        return

    #Code for copying networks if needed.
    def copy_network(self):
        network_copy = Network(self.layers, self.activation_function, self.padding, self.dropout_frequency, self.batch_normalisation, self.game, self.exploration_type, self.data_augmentation)
        network_copy.weights = copy.deepcopy(self.weights)
        network_copy.biases = copy.deepcopy(self.biases)
        network_copy.experiences = copy.deepcopy(self.experiences)
        return network_copy
    
    #Code that allows us to change the exploration strategy of a network after it's been created.
    def set_exploration_strategy(self, new_type, new_parameter): 
        self.exploration_parameter = new_parameter   #Changing epsilon or the exploration temperature.
        self.exploration_type = new_type  #Exploration types available are: 'epsilon-greedy', 'softmax' and 'adaptive-greedy'

In [54]:
#Importing matplotlib for making plots.
import matplotlib.pyplot as plt

#Creating a function for creating plots of networks' accuracies during training.
def plot_accuracies(accuracies, color, yrange, single_epoch = False):
    plt.figure(figsize=(8,5))
    if single_epoch == True:
        barwidth = 1500
        training_examples_seen = np.arange(0, 50001, 2000)
        plt.bar(training_examples_seen, accuracies, barwidth, color=color, edgecolor='black')
        plt.xlabel("Training examples seen", fontsize = 14)
        plt.xlim([0-barwidth, 50000+barwidth])
    else: 
        epochs = np.arange(0,len(accuracies))
        plt.plot(epochs, accuracies, color=color)
        plt.xlabel("Training epochs (i.e. runs over the training set)", fontsize = 14)
        plt.xlim([0, len(accuracies)-1])
        plt.grid()
    #Adding a black horizontal line to mark the 10% accuracy obtainable via random chance.
    plt.plot([-5000, 55000], [10, 10], 'k', linewidth=1.5)
    plt.ylim(yrange)
    #Labelling axes and framing plot window.
    plt.ylabel("% accuracy on unseen test data", fontsize = 14)
    plt.show()
    
#Creating a function for creating a combined plot of three networks' accuracies.
def plot_multi_accuracies(accuracy_array, colors, legend, yrange):
    epochs = np.arange(0,len(accuracies1))
    plt.figure(figsize=(8,4.5))
    plt.grid()
    for (accuracy, color) in zip(accuracy_array, colors):
        plt.plot(epochs, accuracy, color=color)
    plt.legend(legend, loc="best", fontsize=12)
    #Adding a black horizontal line to mark the 10% accuracy obtainable via random chance.
    plt.plot([0, len(accuracy_array[0])-1], [10, 10], 'k', linewidth=1.5)
    #Labelling axes and framing plot window.
    plt.xlim([0, len(accuracy_array[0])-1])
    plt.ylim(yrange)
    plt.xlabel("Training epochs", fontsize = 14)
    plt.ylabel("% accuracy on unseen test data", fontsize = 14)
    plt.show()
    

#Function for converting MNIST image data into the actual image.
def MNIST_plot(data, number, showLabels = False):
    fig, ax = plt.subplots()
    image = np.reshape(data,(28, 28))
    ax.imshow(image, cmap=plt.cm.binary)
    if showLabels == True:
        #Listing our labels (only used if specified in our argument)
        labels = ['Zero', 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine']
        for j in range(10):
            if data[1][j][0] == 1: #Here we check each entry of the vectorised form of our output (i.e. [0, ..., 1, ... 0]) to determine which label we need
                label = labels[j]
        plt.title("Image {:} - {:}".format(number+1,label), fontsize = 15)
    else:
        plt.title("Image {:}".format(number+1), fontsize = 15)
    plt.xticks([], [])
    plt.yticks([], [])
    plt.show()
    
#Function for visualising the weights learned by a convolutional filter.
def filter_plot(filter, number):
    fig, ax = plt.subplots()
    filter_size = int(np.sqrt(np.size(filter)))
    #Reshaping the filter weights into the filter shape.
    image = np.reshape(filter,(filter_size, filter_size))
    ax.imshow(image, cmap=plt.cm.binary)
    plt.title("Filter {:}".format(number), fontsize = 15)
    plt.show()
    return

In [ ]:
#Loading the training and test data.
training_data, test_data = load_data_wrapper()
#Here I also define small and medium versions of the training and test data that can be used for faster but less accurate training.
training_data_small = random.sample(training_data, 500)
training_data_medium = random.sample(training_data, 5000)
test_data_small = random.sample(test_data, 100)
test_data_medium = random.sample(test_data, 1000)